In [ ]:
import h5py
import numpy as np
from blutils.fit_2dgaussians import fit_gaussian_to_imgs
from scipy.optimize import curve_fit
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage import img_as_float
from sklearn.metrics import mean_squared_error

In [ ]:
path = r'C:/Users/alba.gomez.segalas/Documents/MF3 measurements/2024-03-01 beam monitor'
filename = 'AR2-BR2-AR2-BR2.h5'
hf = h5py.File(path + '/' + filename, 'r')

### Implementation for general number of maxima ###

In [ ]:
def img_around_peak(image, peak, width):
    """Return a subimage around the peak, with a given width."""
    row, col = peak
    img = image[row-width//2:row+width//2, col-width//2:col+width//2]
    return img

def create_cpufit_input(data, num_peaks, coordinates, average, width):
    starts = np.arange(0, data.shape[0], average)
    slices = np.array([slice(start, start + average) for start in starts])

    cpufit_input = np.zeros((len(slices)*num_peaks, width, width), dtype=np.float32)

    for i, s in enumerate(slices):
        image = np.mean(data[s], axis=0)
        for j, peak in enumerate(coordinates):
            cpufit_input[i*num_peaks+j] = img_around_peak(image=image, peak=peak, width=width)

    return cpufit_input

In [ ]:
# Select first frame of stack
image = hf['Stacks/movie'][0].astype(np.float32) 
image = ndi.gaussian_filter(image, sigma=3)  

# Define number of peaks in image
num_peaks = 4 

# Find coordinates of peaks
coordinates = peak_local_max(image, min_distance=10, num_peaks=num_peaks)
print("Indices of the highest local maxima:", coordinates)

# def peak_coords(image, num_peaks=4):
#     """Return the coordinates of the highest local maxima in the image."""
#     image = ndi.gaussian_filter(image, sigma=3)  
#     coordinates = peak_local_max(image, min_distance=10, num_peaks=num_peaks)
#     return coordinates

plt.imshow(image)
plt.scatter(coordinates[:, 1], coordinates[:, 0], s=100, c='r', marker='x')

width = 20 # width of squared subimage around the peak

fig, axs = plt.subplots(1,num_peaks,figsize=(10,10))
for i, peak in enumerate(coordinates):
    ax = axs[i]
    ax.imshow(img_around_peak(image, peak, width))
    ax.set_title(f"Peak {i+1}")
plt.show()

### Gaussian CPU fit ###

In [ ]:
# Convert HDF5 dataset to numpy array
data = hf['Stacks/movie'][...].astype(np.float32) 
average = 100
width = 20

cpufit_input = create_cpufit_input(data, num_peaks=num_peaks, coordinates=coordinates, average=average, width=width)
chunk_size = 10
parameters, chi_sq = fit_gaussian_to_imgs(cpufit_input, chunk_size=chunk_size)

### Fit analysis ###

In [ ]:
def two_dimensional_gaussian(x, y, amplitude, x0, y0, sigma_x, sigma_y, offset):
    """
    Compute a two-dimensional Gaussian function.
    
    Parameters:
    - x, y: 1D arrays representing the x and y coordinates
    - amplitude: amplitude of the Gaussian
    - x0, y0: center coordinates of the Gaussian
    - sigma_x, sigma_y: standard deviations in x and y directions
    - offset: amplitude offset
    
    Returns:
    - 2D array representing the computed Gaussian function
    """
    x_term = (x - x0) ** 2 / (2 * sigma_x ** 2)
    y_term = (y - y0) ** 2 / (2 * sigma_y ** 2)
    gaussian = amplitude * np.exp(-(x_term + y_term)) + offset
    return gaussian

def two_dimensional_elliptical_gaussian(x, y, amplitude, x0, y0, sigma_x, sigma_y, offset, angle):
    """
    Compute a two-dimensional elliptical Gaussian function.
    
    Parameters:
    - x, y: 1D arrays representing the x and y coordinates
    - amplitude: amplitude of the Gaussian
    - x0, y0: center coordinates of the Gaussian
    - sigma_x, sigma_y: standard deviations in x and y directions
    - offset: amplitude offset
    - angle: rotation angle in degrees
    
    Returns:
    - 2D array representing the computed elliptical Gaussian function
    """
    angle_rad = angle
    # angle_rad = np.deg2rad(angle)
    x_rot = (x - x0) * np.cos(angle_rad) - (y - y0) * np.sin(angle_rad)
    y_rot = (x - x0) * np.sin(angle_rad) + (y - y0) * np.cos(angle_rad)
    gaussian = amplitude * np.exp(-((x_rot / sigma_x) ** 2 + (y_rot / sigma_y) ** 2) / 2) + offset
    return gaussian
    

In [ ]:
gr = data.shape[0]/average
if isinstance(gr, int):
    pass
else:
    gr = int(gr) + 1
random_gr = np.random.choice(range(gr))

X, Y = np.meshgrid(np.arange(width), np.arange(width))

fig, axs = plt.subplots(3, num_peaks, figsize=(15, 15))

for i, peak in enumerate(coordinates):
    
    raw_peak = cpufit_input[i + num_peaks * random_gr]
    im0 = axs[0][i].imshow(raw_peak)
    axs[0][i].set_title(f"Raw Peak {i+1}")
    
    fit_peak = two_dimensional_gaussian(X, Y, *parameters[i + num_peaks * random_gr][:6])
    im1 = axs[1][i].imshow(fit_peak)
    axs[1][i].set_title(f"Fit Peak {i+1}")
    
    err = np.abs(raw_peak - fit_peak)/np.max(raw_peak)
    im2 = axs[2][i].imshow(err)
    axs[2][i].set_title(f"Residual Peak {i+1}")
    
    fig.colorbar(im0, ax=axs[0][i])
    fig.colorbar(im1, ax=axs[1][i])
    fig.colorbar(im2, ax=axs[2][i])

plt.tight_layout()
plt.show()

# image = raw_peak
# plt.imshow(image)
# max_point = np.unravel_index(np.argmax(image), image.shape)
# plt.plot(max_point[1], max_point[0], 'r+', markersize=10)

In [ ]:
fig, axs = plt.subplots(3, num_peaks, figsize=(15, 15))

for i, peak in enumerate(coordinates):
    
    raw_peak = cpufit_input[i + num_peaks * random_gr]
    im0 = axs[0][i].imshow(raw_peak)
    axs[0][i].set_title(f"Raw Peak {i+1}")
    
    fit_peak = two_dimensional_elliptical_gaussian(X, Y, *parameters[i + num_peaks * random_gr][:7])
    im1 = axs[1][i].imshow(fit_peak)
    axs[1][i].set_title(f"Fit Peak {i+1}")
    
    err = np.abs(raw_peak - fit_peak)/np.max(raw_peak)
    im2 = axs[2][i].imshow(err)
    axs[2][i].set_title(f"Residual Peak {i+1}")
    
    fig.colorbar(im0, ax=axs[0][i])
    fig.colorbar(im1, ax=axs[1][i])
    fig.colorbar(im2, ax=axs[2][i])

plt.tight_layout()
plt.show()

In [ ]:
# 1D plots

### Drift analysis ###

In [ ]:
plt.figure()
plt.plot(parameters[:len(slices), 1], label='x0', linestyle = '-', color='r')
plt.plot(parameters[:len(slices), 2], label='y0', linestyle = ':', color='r')
plt.plot(parameters[len(slices):, 1], label='x1', linestyle = '-', color='k')
plt.plot(parameters[len(slices):, 2], label='y1', linestyle = ':', color='k')
plt.legend()
plt.show()

#TODO plot (x,y) trajectories in 2D plot for each peak, encode time in trace color 

### Test subparts of CPU fitting code 

In [ ]:
nfit = 120
chunk_size = 50
np.linspace(0,nfit,np.ceil(nfit/chunk_size).astype(int)+1)